# First-run walkthrough (3 steps)

This compact notebook is a 3-cell sequence intended to be run top-to-bottom in Colab or locally. It demonstrates: mounting Drive (optional), composing a minimal flow JSON using the project's `composer` helper, and showing how to start cloudflared in background (if you choose to).

Run each cell and follow prompts. Cells are safe by default; where an action starts a process (cloudflared) you must uncomment the execution line to run it.

In [ ]:
# Cell 1 — Mount Drive (optional) and set DRIVE_ROOT
import os
from pathlib import Path
DRIVE_ROOT = Path(os.environ.get('DRIVE_ROOT', '/content/drive/MyDrive/ComfyUI'))
print('Current DRIVE_ROOT:', DRIVE_ROOT)
try:
    import google.colab
    from google.colab import drive
    print('Running in Colab. If Drive is not mounted, uncomment the next line to mount interactively.')
    # drive.mount('/content/drive')
except Exception:
    print('Not in Colab or google.colab not available — using local DRIVE_ROOT')
DRIVE_ROOT.mkdir(parents=True, exist_ok=True)
print('DRIVE_ROOT ready at', DRIVE_ROOT)

In [ ]:
# Cell 2 — Compose a minimal flow using Helper Scipts/composer.py
from pathlib import Path
import importlib.util
import json

def import_module_from_path(module_name, path):
    spec = importlib.util.spec_from_file_location(module_name, str(Path(path)))
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

composer_path = Path('Helper Scipts') / 'composer.py'
if composer_path.exists():
    composer = import_module_from_path('composer', composer_path)
    out_path, flow = composer.compose_flow('Quick walkthrough prompt', model_key=None)
    print('Composed flow at:', out_path)
    print(json.dumps(flow, indent=2)[:1000])
else:
    print('composer.py not found — check the repository layout (Helper Scipts/)')

In [ ]:
# Cell 3 — Start cloudflared in background (optional action)
# This cell shows the exact command to background cloudflared with nohup and log to /tmp/cloudflared.log.
# It is commented out by default — uncomment to run in Colab after ensuring cloudflared is installed.
import os
cmd = 'nohup cloudflared tunnel --url http://localhost:8188 > /tmp/cloudflared.log 2>&1 &'
print('Background cloudflared command (copy/paste or run in a shell cell):')
print(cmd)
print('To run in Colab (shell cell):')
print('!' + cmd)
# Uncomment to run directly in this cell (only do this if you have cloudflared installed and want to run it):
# import subprocess
# subprocess.check_output(cmd, shell=True)
print('Logs will be written to /tmp/cloudflared.log — monitor with: !tail -n 200 /tmp/cloudflared.log')